In [ ]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import movingpandas as mpd
from shapely.geometry import box
from matplotlib.animation import FFMpegWriter
from datetime import timedelta
import colorcet as cc

In [ ]:
states_provinces_gdf = gpd.read_file("../../natural_earth_vector/10m_cultural/ne_10m_admin_1_states_provinces.shp")
aus_geometry = states_provinces_gdf[states_provinces_gdf["admin"] == "Australia"].unary_union

In [ ]:
min_lon = 110.0
max_lon = 157.0
min_lat = -45.0
max_lat = -5.0
australia_bbox = box(min_lon, min_lat, max_lon, max_lat)

In [ ]:
aus_poly_geoseries = gpd.GeoSeries([aus_geometry])

In [ ]:
ax = aus_poly_geoseries.plot()
# ax.set_facecolor("black")
ax.set_xlim(min_lon, max_lon)
ax.set_ylim(min_lat, max_lat)

In [ ]:
vessel_files = [
    f"Vessel_Traffic_Data_{date:%B_%Y}.parquet" for date in pd.date_range(start="Nov 2022", end="Nov 2023", freq="M")
]

In [ ]:
vessel_pos_gdf_list = [
    gpd.read_parquet(Path("../scrape_amsa_vessel_positions/vessel_position_parquets") / file)
    for file in tqdm(vessel_files)
]

In [ ]:
vessel_pos_combined_gdf = pd.concat(vessel_pos_gdf_list, axis=0, ignore_index=True)

In [ ]:
vessel_pos_combined_gdf.to_parquet("vessel_pos_combined.parquet")

In [ ]:
vessel_pos_combined_gdf = gpd.read_parquet("vessel_pos_combined.parquet")

In [ ]:
vessel_pos_combined_gdf.drop(["COURSE", "SPEED"], axis=1, inplace=True)

In [ ]:
ship_traj_collection = mpd.TrajectoryCollection(
    vessel_pos_combined_gdf, traj_id_col="CRAFT_ID", t="TIMESTAMP", x="LON", y="LAT"
)

In [ ]:
clipped_ship_traj_collection = ship_traj_collection.clip(australia_bbox)

In [ ]:
split_ship_traj_collection = mpd.ObservationGapSplitter(clipped_ship_traj_collection).split(gap=timedelta(days=5))

In [ ]:
resampled_ship_trajectories = []
for ship_traj in tqdm(split_ship_traj_collection.trajectories):
    df = ship_traj.df.drop(columns="geometry")
    resampled_df = df[["LON", "LAT"]].resample("60T").first().interpolate()
    if len(resampled_df) < 2:
        continue
    additional_columns = [column for column in df.columns if column not in {"LON", "LAT"}]
    for column in additional_columns:
        resampled_df[column] = df[column].iloc[0]
    craft_id = resampled_df["CRAFT_ID"].iloc[0].split("_")[0]
    resampled_ship_trajectories.append(mpd.Trajectory(resampled_df, craft_id, traj_id_col="CRAFT_ID", x="LON", y="LAT"))

In [ ]:
line_str_ship_trajectory_list = [ship_traj.to_line_gdf() for ship_traj in tqdm(resampled_ship_trajectories)]

In [ ]:
line_str_ship_trajectory_gdf = pd.concat(line_str_ship_trajectory_list, axis=0, ignore_index=True)

In [ ]:
line_str_ship_trajectory_gdf.to_parquet("ship_lines.parquet")

In [ ]:
line_str_ship_trajectory_gdf = gpd.read_parquet("ship_lines.parquet")

In [ ]:
gdf_index = line_str_ship_trajectory_gdf.sindex

In [ ]:
land_intersecting_lines = line_str_ship_trajectory_gdf.intersects(aus_geometry)

In [ ]:
land_intersecting_lines.sum()

In [ ]:
filtered_ship_lines_gdf = line_str_ship_trajectory_gdf[~land_intersecting_lines]

In [ ]:
filtered_ship_lines_gdf.to_parquet("filtered_ship_lines.parquet")

In [ ]:
filtered_ship_lines_gdf = gpd.read_parquet("filtered_ship_lines.parquet")

In [ ]:
gdf = filtered_ship_lines_gdf.to_crs("EPSG:3577")
geometry_lengths = filtered_ship_lines_gdf.geometry.length

In [ ]:
geometry_lengths.quantile([0.05, 0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 0.9999, 0.99999, 0.999999])

In [ ]:
len(geometry_lengths)

In [ ]:
DPI = 200
FRAME_RATE = 60
anim_file_path = Path("./ship_movement.mp4")
file_writer = FFMpegWriter(fps=FRAME_RATE)
tail_length = 12
pallette = cc.fire
pallette_max_index = len(pallette) - 1
time_delta_colours = [
    (pd.Timedelta(hours=step), pallette[pallette_max_index - int((step / tail_length) * pallette_max_index)])
    for step in range(tail_length, -1, -1)
]
time_index = pd.date_range(start=filtered_ship_lines_gdf["t"].min(), end=filtered_ship_lines_gdf["t"].max(), freq="H")

In [ ]:
fig = plt.figure(facecolor="black", dpi=DPI)
ax = fig.add_axes([0, 0, 1, 1])
clock_axes = fig.add_axes([0.75, 0.85, 0.23, 0.13], facecolor=None)
with file_writer.saving(fig, anim_file_path, dpi=DPI):
    for time in tqdm(time_index):
        ax.cla()
        clock_axes.cla()
        aus_poly_geoseries.plot(color="darkorange", ax=ax, aspect=None, zorder=1)
        for time_delta, color in time_delta_colours:
            ship_mask = filtered_ship_lines_gdf["t"] == (time - time_delta)
            if ship_mask.sum() > 0:
                filtered_ship_lines_gdf[ship_mask].plot(color=color, linewidth=0.5, ax=ax, aspect=None, zorder=2)
        ax.set_xlim(min_lon, max_lon)
        ax.set_ylim(min_lat, max_lat)
        ax.set_axis_off()
        clock_axes.set_axis_off()
        clock_axes.text(
            0.5,
            0.5,
            f"Vessel Positions\n{time:%d %b %Y %H:%M} UTC",
            fontsize=10,
            ha="center",
            color="white",
            bbox={"boxstyle": "round", "alpha": 0.5, "facecolor": "salmon"},
        )
        file_writer.grab_frame()